In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/metro_rush.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,rush_hour,metro_dist,temp,hum,wind,member_type,holiday_1,work_day_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
278164,6.817,0.585465,0,0.023451,19.142500,78.7917,8.500357,1,0,1,...,0,1,0,0,0,0,0,1,0,0
666376,10.183,1.181485,0,0.048442,27.132500,57.8333,12.292557,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1175404,2.483,0.304332,0,0.181761,16.988349,82.1250,6.000406,1,0,1,...,0,0,0,1,0,0,0,0,1,0
221214,18.550,0.000000,0,0.761098,25.605000,30.5000,19.583229,1,0,1,...,0,0,0,0,1,0,0,1,0,0
961020,8.033,1.070473,0,0.535903,6.508712,64.6522,12.565984,1,0,1,...,0,0,1,0,0,0,0,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [6]:
# Remove Lasso Columns while including all of the start/end stations
remove_cols = ['weekday_0','weather_cat_2','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [15]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((914885, 295), (304962, 295), (914885,), (304962,))

In [16]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [18]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([  4.13462334e-01,   4.20877836e-01,   4.15300117e-01,
         -1.99177456e+10,   4.13631457e-01]),
 0.41591969054485123,
 0.41535414108646318,
 79.706417383253239,
 8.9278450582015161)

In [11]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     2259.
Date:                Thu, 14 Dec 2017   Prob (F-statistic):               0.00
Time:                        21:10:57   Log-Likelihood:            -3.3041e+06
No. Observations:              914885   AIC:                         6.609e+06
Df Residuals:                  914596   BIC:                         6.612e+06
Df Model:                         288                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             6.1966      0.013    461.493      0.000       6.170       6.223
x2            -1.4811      0.029    -50.345      0.000      -1.539      -1.423
x3             0.9827      0.175      5.612      0.000       0.639       1.326
x4             0.0563      0.003     19.759      0.000       0.051       0.062
x5            -0.0049      0.001     -4.816      0.000      -0.007      -0.003
x6            -0.0176      0.002     -8.405      0.000      -0.022      -0.013
x7           -11.2396      0.028   -407.722      0.000     -11.294     -11.186
x8            -0.3186      0.059     -5.428      0.000      -0.434      -0.204
x9            -1.0459      0.024    -43.670      0.000      -1.093      -0.999
x10            0.1024      0.208      0.492      0.623      -0.306       0.511
x11           -0.5430      0.103     -5.279      0.000      -0.745      -0.341
x12            3.9037      0.100     39.151      0.000       3.708       4.099
x13           -0.9591      0.244     -3.927      0.000      -1.438      -0.480
x14           -1.3777      0.096    -14.385      0.000      -1.565      -1.190
x15            1.8148      0.304      5.980      0.000       1.220       2.410
x16            0.1508      0.316      0.477      0.634      -0.469       0.771
x17           -1.5217      0.123    -12.366      0.000      -1.763      -1.281
x18           -1.1007      0.144     -7.632      0.000      -1.383      -0.818
x19            0.6638      0.102      6.506      0.000       0.464       0.864
x20            3.6109      0.116     31.199      0.000       3.384       3.838
x21           -0.5020      0.122     -4.120      0.000      -0.741      -0.263
x22            1.5375      0.106     14.562      0.000       1.331       1.744
x23           -1.8301      0.094    -19.527      0.000      -2.014      -1.646
x24           -0.9319      0.090    -10.310      0.000      -1.109      -0.755
x25           -0.7066      0.087     -8.139      0.000      -0.877      -0.536
x26           -0.8664      0.082    -10.523      0.000      -1.028      -0.705
x27           -1.8843      0.124    -15.165      0.000      -2.128      -1.641
x28           -1.7504      0.230     -7.612      0.000      -2.201      -1.300
x29            3.4831      0.224     15.569      0.000       3.045       3.922
x30           -0.2624      0.078     -3.367      0.001      -0.415      -0.110
x31           -1.3641      0.272     -5.024      0.000      -1.896      -0.832
x32           -1.6646      0.086    -19.437      0.000      -1.832      -1.497
x33           -0.3150      0.076     -4.163      0.000      -0.463      -0.167
x34            0.6316      0.115      5.482      0.000       0.406       0.857
x35           -0.1754      0.095     -1.855      0.064      -0.361       0.010
x36            0.2593      0.200      1.300      0.194      -0.132       0.650
x3

### Model 4 - Ridge with Round 1

In [12]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [13]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [14]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.41656522092613768,
 79.414202372003487,
 8.9114646591906261,
 0.41600029651768122)